In [1]:
#import sys
#print(sys.executable) = environment variable PYSPARK_PYTHON

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tools').getOrCreate()

In [2]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer

In [3]:
df_sentence = spark.createDataFrame([(0.0, 'Hi i heard about Spark'),
                                     (0.0, 'I wish java could use case classes'),
                                     (1.0, 'Logistic regression models are neat')],
                                    ['label', 'sentence'])

df_sentence.printSchema()

root
 |-- label: double (nullable = true)
 |-- sentence: string (nullable = true)



In [4]:
df_sentence.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi i heard about ...|
|  0.0|I wish java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [5]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [6]:
words_data = tokenizer.transform(df_sentence)

In [7]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi i heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [8]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [9]:
featurized_data = hashing_tf.transform(words_data)

In [10]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [11]:
idf_model = idf.fit(featurized_data)

In [12]:
rescaled_data = idf_model.transform(featurized_data)

In [13]:
rescaled_data.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262144,[18700,19...|
|  0.0|(262144,[19036,20...|
|  1.0|(262144,[46243,58...|
+-----+--------------------+



In [14]:
df = spark.createDataFrame([(0, "a b c".split(" ")),
                            (1, "a b b c a".split(" "))],
                           ["id", "words"])

In [15]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [16]:
cv = CountVectorizer(inputCol='words', outputCol='feature', vocabSize=3, minDF=2.0)

In [17]:
model = cv.fit(df)

In [18]:
result = model.transform(df)

In [19]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |feature                  |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

